In [1]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

1.11.0


/home/antonio/anaconda3/envs/geometric_new/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
from torch_geometric.utils import train_test_split_edges

# Tutorial 6  
Graph AutoEncoders GAE &  
Variational Graph Autoencoders VGAE    

[paper](https://arxiv.org/pdf/1611.07308.pdf)  
[code](https://github.com/rusty1s/pytorch_geometric/blob/master/examples/autoencoder.py)

## Graph AutoEncoder GAE

### Load the data

In [3]:
dataset = Planetoid("\..", "CiteSeer", transform=T.NormalizeFeatures())
dataset.data

Processing...
Done!


Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])

In [4]:
data = dataset[0]
data.train_mask = data.val_mask = data.test_mask = None
data

Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327])

In [5]:
data = train_test_split_edges(data)

/home/antonio/anaconda3/envs/geometric_new/lib/python3.9/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


In [6]:
data

Data(x=[3327, 3703], y=[3327], val_pos_edge_index=[2, 227], test_pos_edge_index=[2, 455], train_pos_edge_index=[2, 7740], train_neg_adj_mask=[3327, 3327], val_neg_edge_index=[2, 227], test_neg_edge_index=[2, 455])

### Define the Encoder

In [7]:
class GCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels, cached=True) # cached only for transductive learning
        self.conv2 = GCNConv(2 * out_channels, out_channels, cached=True) # cached only for transductive learning

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)
    

### Define the Autoencoder

In [8]:
from torch_geometric.nn import GAE

In [9]:
# parameters
out_channels = 2
num_features = dataset.num_features
epochs = 100

# model
model = GAE(GCNEncoder(num_features, out_channels))

# move to GPU (if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)

# inizialize the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [10]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    #if args.variational:
    #   loss = loss + (1 / data.num_nodes) * model.kl_loss()
    loss.backward()
    optimizer.step()
    return float(loss)


def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)



In [11]:
for epoch in range(1, epochs + 1):
    loss = train()

    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))

Epoch: 001, AUC: 0.6280, AP: 0.6691
Epoch: 002, AUC: 0.6498, AP: 0.6876
Epoch: 003, AUC: 0.6555, AP: 0.6970
Epoch: 004, AUC: 0.6585, AP: 0.6971
Epoch: 005, AUC: 0.6600, AP: 0.6978
Epoch: 006, AUC: 0.6608, AP: 0.7002
Epoch: 007, AUC: 0.6614, AP: 0.7029
Epoch: 008, AUC: 0.6616, AP: 0.7038
Epoch: 009, AUC: 0.6620, AP: 0.7039
Epoch: 010, AUC: 0.6624, AP: 0.7045
Epoch: 011, AUC: 0.6624, AP: 0.7055
Epoch: 012, AUC: 0.6624, AP: 0.7065
Epoch: 013, AUC: 0.6626, AP: 0.7078
Epoch: 014, AUC: 0.6627, AP: 0.7090
Epoch: 015, AUC: 0.6627, AP: 0.7105
Epoch: 016, AUC: 0.6617, AP: 0.7115
Epoch: 017, AUC: 0.6613, AP: 0.7135
Epoch: 018, AUC: 0.6601, AP: 0.7145
Epoch: 019, AUC: 0.6589, AP: 0.7155
Epoch: 020, AUC: 0.6571, AP: 0.7160
Epoch: 021, AUC: 0.6562, AP: 0.7170
Epoch: 022, AUC: 0.6558, AP: 0.7180
Epoch: 023, AUC: 0.6561, AP: 0.7193
Epoch: 024, AUC: 0.6559, AP: 0.7196
Epoch: 025, AUC: 0.6562, AP: 0.7204
Epoch: 026, AUC: 0.6569, AP: 0.7211
Epoch: 027, AUC: 0.6574, AP: 0.7218
Epoch: 028, AUC: 0.6585, AP:

In [12]:
Z = model.encode(x, train_pos_edge_index)
Z

tensor([[-0.3415,  0.3505],
        [ 0.8631, -1.1042],
        [-0.7020,  0.8189],
        ...,
        [ 0.0874, -0.0409],
        [-0.6144,  0.7314],
        [-0.6832,  0.7997]], device='cuda:0', grad_fn=<AddBackward0>)

## Are the results (AUC) and (AP) easy to read and compare?

# Use Tensorboard

In [17]:
from torch.utils.tensorboard import SummaryWriter

In [18]:
# parameters
out_channels = 2
num_features = dataset.num_features
epochs = 100

# model
model = GAE(GCNEncoder(num_features, out_channels))

# move to GPU (if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)

# inizialize the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

### Import tensorboard

#### Installation: (if needed) "pip install tensorboard"

In [19]:
writer = SummaryWriter('runs/GAE1_experiment_'+'2d_100_epochs')

In [20]:
for epoch in range(1, epochs + 1):
    loss = train()
    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))
    
    
    writer.add_scalar('auc train',auc,epoch) # new line
    writer.add_scalar('ap train',ap,epoch)   # new line

Epoch: 001, AUC: 0.6298, AP: 0.6217
Epoch: 002, AUC: 0.6272, AP: 0.6813
Epoch: 003, AUC: 0.6465, AP: 0.6944
Epoch: 004, AUC: 0.6518, AP: 0.6967
Epoch: 005, AUC: 0.6514, AP: 0.7032
Epoch: 006, AUC: 0.6478, AP: 0.7085
Epoch: 007, AUC: 0.6490, AP: 0.7121
Epoch: 008, AUC: 0.6537, AP: 0.7146
Epoch: 009, AUC: 0.6570, AP: 0.7146
Epoch: 010, AUC: 0.6574, AP: 0.7163
Epoch: 011, AUC: 0.6569, AP: 0.7187
Epoch: 012, AUC: 0.6565, AP: 0.7201
Epoch: 013, AUC: 0.6569, AP: 0.7207
Epoch: 014, AUC: 0.6574, AP: 0.7213
Epoch: 015, AUC: 0.6575, AP: 0.7222
Epoch: 016, AUC: 0.6571, AP: 0.7228
Epoch: 017, AUC: 0.6570, AP: 0.7236
Epoch: 018, AUC: 0.6569, AP: 0.7242
Epoch: 019, AUC: 0.6572, AP: 0.7245
Epoch: 020, AUC: 0.6572, AP: 0.7251
Epoch: 021, AUC: 0.6574, AP: 0.7253
Epoch: 022, AUC: 0.6585, AP: 0.7261
Epoch: 023, AUC: 0.6599, AP: 0.7268
Epoch: 024, AUC: 0.6618, AP: 0.7278
Epoch: 025, AUC: 0.6636, AP: 0.7284
Epoch: 026, AUC: 0.6667, AP: 0.7300
Epoch: 027, AUC: 0.6710, AP: 0.7316
Epoch: 028, AUC: 0.6754, AP:

## Graph Variational AutoEncoder (GVAE)

In [21]:
from torch_geometric.nn import VGAE

In [22]:
dataset = Planetoid("\..", "CiteSeer", transform=T.NormalizeFeatures())
data = dataset[0]
data.train_mask = data.val_mask = data.test_mask = data.y = None
data = train_test_split_edges(data)


class VariationalGCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(VariationalGCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels, cached=True) # cached only for transductive learning
        self.conv_mu = GCNConv(2 * out_channels, out_channels, cached=True)
        self.conv_logstd = GCNConv(2 * out_channels, out_channels, cached=True)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)

/home/antonio/anaconda3/envs/geometric_new/lib/python3.9/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


In [23]:
out_channels = 2
num_features = dataset.num_features
epochs = 300


model = VGAE(VariationalGCNEncoder(num_features, out_channels))  # new line

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [24]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    
    loss = loss + (1 / data.num_nodes) * model.kl_loss()  # new line
    loss.backward()
    optimizer.step()
    return float(loss)


def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)

In [25]:
writer = SummaryWriter('runs/VGAE_experiment_'+'2d_100_epochs')

for epoch in range(1, epochs + 1):
    loss = train()
    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))
    
    
    writer.add_scalar('auc train',auc,epoch) # new line
    writer.add_scalar('ap train',ap,epoch)   # new line

Epoch: 001, AUC: 0.6043, AP: 0.6264
Epoch: 002, AUC: 0.4937, AP: 0.5243
Epoch: 003, AUC: 0.4943, AP: 0.5001
Epoch: 004, AUC: 0.5013, AP: 0.5079
Epoch: 005, AUC: 0.5011, AP: 0.5006
Epoch: 006, AUC: 0.5022, AP: 0.5011
Epoch: 007, AUC: 0.5022, AP: 0.5011
Epoch: 008, AUC: 0.5022, AP: 0.5011
Epoch: 009, AUC: 0.4978, AP: 0.5007
Epoch: 010, AUC: 0.4978, AP: 0.5007
Epoch: 011, AUC: 0.4978, AP: 0.5007
Epoch: 012, AUC: 0.5011, AP: 0.5011
Epoch: 013, AUC: 0.5011, AP: 0.5011
Epoch: 014, AUC: 0.5011, AP: 0.5011
Epoch: 015, AUC: 0.5011, AP: 0.5011
Epoch: 016, AUC: 0.5011, AP: 0.5011
Epoch: 017, AUC: 0.5011, AP: 0.5011
Epoch: 018, AUC: 0.5011, AP: 0.5011
Epoch: 019, AUC: 0.5011, AP: 0.5011
Epoch: 020, AUC: 0.5011, AP: 0.5011
Epoch: 021, AUC: 0.5011, AP: 0.5011
Epoch: 022, AUC: 0.4989, AP: 0.4995
Epoch: 023, AUC: 0.4989, AP: 0.4995
Epoch: 024, AUC: 0.4989, AP: 0.4995
Epoch: 025, AUC: 0.4989, AP: 0.4995
Epoch: 026, AUC: 0.4989, AP: 0.4995
Epoch: 027, AUC: 0.4989, AP: 0.4995
Epoch: 028, AUC: 0.4989, AP:

Epoch: 231, AUC: 0.4989, AP: 0.4995
Epoch: 232, AUC: 0.4989, AP: 0.4995
Epoch: 233, AUC: 0.4989, AP: 0.4995
Epoch: 234, AUC: 0.4989, AP: 0.4995
Epoch: 235, AUC: 0.4989, AP: 0.4995
Epoch: 236, AUC: 0.4989, AP: 0.4995
Epoch: 237, AUC: 0.4989, AP: 0.4995
Epoch: 238, AUC: 0.4989, AP: 0.4995
Epoch: 239, AUC: 0.4989, AP: 0.4995
Epoch: 240, AUC: 0.4989, AP: 0.4995
Epoch: 241, AUC: 0.5011, AP: 0.5011
Epoch: 242, AUC: 0.5011, AP: 0.5011
Epoch: 243, AUC: 0.5011, AP: 0.5011
Epoch: 244, AUC: 0.5011, AP: 0.5011
Epoch: 245, AUC: 0.5011, AP: 0.5011
Epoch: 246, AUC: 0.5011, AP: 0.5011
Epoch: 247, AUC: 0.5011, AP: 0.5011
Epoch: 248, AUC: 0.5011, AP: 0.5011
Epoch: 249, AUC: 0.5011, AP: 0.5011
Epoch: 250, AUC: 0.5011, AP: 0.5011
Epoch: 251, AUC: 0.5011, AP: 0.5011
Epoch: 252, AUC: 0.5011, AP: 0.5011
Epoch: 253, AUC: 0.5011, AP: 0.5011
Epoch: 254, AUC: 0.5011, AP: 0.5011
Epoch: 255, AUC: 0.5011, AP: 0.5011
Epoch: 256, AUC: 0.5011, AP: 0.5011
Epoch: 257, AUC: 0.5011, AP: 0.5011
Epoch: 258, AUC: 0.5011, AP: